In [6]:
#First we load the data and packages.
import pandas as pd
import numpy as np
from copy import deepcopy
from scipy.spatial.distance import pdist, squareform
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
# the command below only needs to be run once
#import sys
#!{sys.executable} -m pip install xgboost
import xgboost as xgb
#from xgboost.xgbclassifier import XGBClassifier as xgb

In [7]:
# load train and test data, make dummies from genres and combine it with df.
df1 = pd.read_csv("../../Data/ml-latest-small/PreprocessedData_ml_latest_year_small.csv",index_col=0)
genres = df1.genres.str.get_dummies(sep = "|")

df = pd.concat([df1, genres], axis=1)
df = df.drop(['genres'], axis = 1)

In [22]:
#create features and target data
data = df.drop(['rating', 'title'], axis= 1)
target = df[['rating']]

In [13]:
#split df in training and test data
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state= 101)

In [23]:
#build a content-based model with XGBoost
model1 = xgb.XGBRegressor(objective='reg:squarederror')
model1.fit(X_train, y_train) 

pred1 = model1.predict(X_test)
rmse = np.sqrt(np.mean((pred1 - y_test.to_numpy())**2))
print(f'content-based rmse = {rmse}')



content-based rmse = 1.1594521885467903
